<center>

# Проект: <br/> Бриф учебного кейса «Модель прогнозирования стоимости жилья для агентства недвижимости»


</center>

Импортируем необходимые библиотеки для работы

In [2]:
# Импортируем pandas для работы с DataFrame
import pandas as pd
# Импортируем numpy для вычислений
import numpy as np

# Регулярные выражения
import re

import json

# Счетчик
from collections import Counter

# Для визуализации
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


from scipy import stats
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import cluster
from sklearn import feature_selection


## Данные

Ознакомимся с данными и рассмотрим их.

In [3]:
# Прочитаем данные 
# и выведим первые несколько строк таблицы для проверки, что все считалось как надо
data = pd.read_csv('data/data.csv')
display(data.head())

# Размеры датафреймы
print('Кол-во записей: {}'.format(data.shape[0]))
print('Кол-во признаков: {}'.format(data.shape[1]))

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


Кол-во записей: 377185
Кол-во признаков: 18


In [4]:
# Копия первоначального датафрейма
data_primal = data.copy()

Ознакомимся с информацией о признаках в данных

In [5]:
# Зафиксируем список столбцов датафрейма который имеем изначально
columns_names_primal = data.columns
# Список для названий признаков которые будут удалены
drop_list = []
# информация
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

> В данных можно наблюдать, что в разшличных признаках имеются различное написание одного и тогоже (через "-", с разным регистром и прочее).

> Все признаки имеют тип object

> Наблюдаем наличие 'PrivatePool' и 'private pool'. Можно предтположиь, что это обусловленно тем, что один из признаков (скорее всего 'PrivatePool') был добавлен и дублирует значения или в нем можно найти добавленные значения.

> Наблюдается множество пропусков - фактически во всех столбцах кроме state, zipcode, schools, homeFacts. <br/><br/> Очень большое значение пропусков имеется в признаках private pool / PrivatePool, mls-id, fireplace.

Раскроем признаки "homeFacts" и "schools"

In [6]:
# Определим функцию отбора признаков для результата выполнения функции
def select_feature(func, options_feature=None):
    def wrap(*args,**kwargs):
        df = func(*args,**kwargs)
        if options_feature != None:
            df = df[options_feature]
            return df
        else:
            return df
    
    return wrap


# Определим функцию сериализации и получения датафрейма с расширенными данными по признакам
def serialize_fact(dataframe):
    # Определяем сеарилизацию - получение объекта
    def serialize(s):
        s = re.sub(r":\s[\w\d]+\W", ": '',", s) 
        s = re.sub(r"\w+(?:')\w", "`", s)
        s = re.sub(r"'", '"',s)
        s = re.sub(r'(?:(?<!{)(?<!:\s)(?<!,\s))"(?:(?!})(?!:)(?!,\s))', '`', s)
        facts = json.loads(s)
        
        return facts

    # Делаем датафрем расширения
    df = dataframe[['homeFacts']].copy()
    # Производм сеарилизацию
    df['homeFacts']=df['homeFacts'].apply(lambda x: serialize(x))
    
    # Через цикл построчно производим присвоение значений по указателям
    for row in df.iterrows():
        for fact in row[1]:
            for el in fact['atAGlanceFacts']:
                df.loc[row[0], el['factLabel']] = el['factValue']
    
    # Приведем пустые строки к NaN
    for column in df.columns:
        mask = (df[column]=='')
        idx = df[mask][column].index
        df.loc[idx, column] = np.NaN
    
    return df

# Определяем функцию сериализации школ
def serialize_schools(dataframe):
    # Определяем сеарилизацию - получение объекта
    def serialize(s):
        s = re.sub(r"None", " '',", s) 
        s = re.sub(r"'", '"',s)
        s = re.sub(r'(?:(?<!{)(?<!\[)(?<!:\s)(?<!,\s))"(?:(?!})(?!\])(?!:)(?!,\s))', '`', s)
        s = re.sub(r', "Grades".*', '}}]', s)
        facts = json.loads(s)
        
        return facts

    # Делаем датафрем расширения
    df = dataframe[['schools']].copy()
    # Производм сеарилизацию
    df['schools']=df['schools'].apply(lambda x: serialize(x))
    
    # Через цикл построчно производим присвоение значений по указателям
    def to_numeric(x):
        if x is None:
            return 0
        x = re.search(r'(?:(\d+\.\d+)|(\d+))', x).group()
        return float(x)
    
    for row in df.iterrows():
        for schools in row[1]:
            # Берем в цилке перебор елементов
            for el in schools:
                # Берем значение дистанции
                distance = el['data']['Distance']
                distance_arr = np.array(list(map(to_numeric, distance)))
                # Берем минимальную дистанцию и ее индекс
                distance_arg = distance_arr.argmin() if distance else -1
                # Записываем в датафрем значения рейтинга и дистанции по минимальной
                df.loc[row[0], 'school_rating'] = el['rating'][distance_arg] if distance_arg != -1 else 0
                df.loc[row[0], 'school_distance'] = distance_arr[distance_arg] if distance_arg != -1 else 0
                
    return df

In [7]:
data_extension_schools = data.copy()

data_extension_schools = serialize_schools(data_extension_schools)
data_extension_schools

,schools,school_rating,school_distance
0,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",4,2.70
1,"[{'rating': ['4/10', ' ``,/10', '4/10'], 'data...",4/10,1.01
2,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...",8/10,1.19
3,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...",9/10,0.10
4,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",5/10,3.03
...,...,...,...
377180,"[{'rating': ['10/10', '5/10'], 'data': {'Dista...",5/10,1.10
377181,"[{'rating': ['1/10', '5/10', '7/10'], 'data': ...",7/10,0.40
377182,"[{'rating': ['5/10', '4/10'], 'data': {'Distan...",5/10,0.48
377183,"[{'rating': ['NA', 'NA', 'NA'], 'data': {'Dist...",NA,0.50


In [8]:
data_extension = data.copy()

data_extension = serialize_fact(data_extension)
data_extension

,homeFacts,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,Price/sqft
0,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",2019,NaN,"Central A/C, Heat Pump",NaN,NaN,NaN,$144
1,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",2019,NaN,NaN,NaN,NaN,5828 sqft,$159/sqft
2,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",1961,1967,Forced Air,Central,Attached Garage,"8,626 sqft",$965/sqft
3,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",2006,2006,Forced Air,Central,Detached Garage,"8,220 sqft",$371/sqft
4,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,NaN,NaN,NaN,NaN,"10,019 sqft",NaN
...,...,...,...,...,...,...,...,...
377180,"{'atAGlanceFacts': [{'factValue': '1990', 'fac...",1990,1990,Other,Central,2 spaces,"8,500 sqft",$311
377181,"{'atAGlanceFacts': [{'factValue': '1924', 'fac...",1924,NaN,Radiant,NaN,None,NaN,$337/sqft
377182,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",1950,1950,Other,NaN,2,"1,600 sqft",$458/sqft
377183,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


Уберем в значениях признаков разный регист и лишние пробелы. И заменим разделение '/' и обозначение чисел словами

In [9]:
# Определим функцию приведения к нижнему регистру и замен
def to_general_form(data, features:list, features_2:list=None):
    '''
    Функция принимает датафрейм и приозвольный список признаков данного датафрейма. 
    Для каждого признака из списка в датафрему производится смена регистра значений как строк
    И производится замена значений разделителей и замена обозначения чисел.
    Возращается измененный датафрейм
    '''
    # Определяем данные
    df = data.copy()
    # Определяем словарь замены чисел обозначеных словом к числам
    replace_values = {
        '1': 'one',
        '2': 'two',
        '3': 'three',
        '4': 'four',
        '5': 'five',
        '6': 'six', 
        '7': 'seven'
    }
    # В цикле проходим по признакам и в каждом делаем замены 
    for feature in features:
        df[feature] = df[feature]\
            .str.lower().str.strip()\
            .str.replace(',', ', ')\
            .str.replace('/', ' / ')
        df[feature] = df[feature].apply(lambda x: np.NaN if x is np.NaN else ' '.join(x.split()))
        for key, value in replace_values.items():
            df[feature] = df[feature].apply(lambda x: np.NaN if x is np.NaN else re.sub(fr'\b{value}\b', key, x))
            df[feature] = df[feature].apply(lambda x: np.NaN if x is np.NaN else re.sub(r'\b\d-', key+' ', x))
                
    if features_2 is not None:           
        for feature in features_2:
            df[feature] = df[feature]\
                .str.lower().str.strip()\
                .str.replace('/', ' / ')
            df[feature] = df[feature].apply(lambda x: np.NaN if x is np.NaN else ' '.join(x.split()))
            for key, value in replace_values.items():
                df[feature] = df[feature].apply(lambda x: np.NaN if x is np.NaN else re.sub(fr'\b{value}\b', key, x))
                df[feature] = df[feature].apply(lambda x: np.NaN if x is np.NaN else re.sub(r'\b\d-', key+' ', x))

    return df

In [10]:
# Списки признаков
features = [
    'status', 'propertyType', 'private pool', 'PrivatePool',
    'fireplace', 'mls-id', 'MlsId', 'street', 'state', 'city']
features_2 = ['beds', 'baths', 'sqft', 'stories']

# Выпоним приведене данных к форме
data = to_general_form(data, features, features_2)
display(data.head(5))

# Списки признаков
mask = data_extension.columns != 'homeFacts'
features = data_extension.columns[mask]
# Выпоним приведене данных к форме
data_extension = to_general_form(data_extension, features)
display(data_extension.head(5))

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,active,NaN,single family home,240 heather ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",gas logs,southern pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,nc,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 e heroy ave,3 baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,spokane valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 beds,wa,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 westridge rd,2 baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,los angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 beds,ca,1.0,NaN,yes,fr19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 livingston ave,8 baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 beds,tx,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot / land,1524 kiscoe st,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,palm bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,fl,NaN,NaN,NaN,861745,"$5,000"


,homeFacts,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,Price/sqft
0,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",2019,NaN,"central a / c, heat pump",NaN,NaN,NaN,$144
1,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",2019,NaN,NaN,NaN,NaN,5828 sqft,$159 / sqft
2,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",1961,1967,forced air,central,attached garage,"8, 626 sqft",$965 / sqft
3,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",2006,2006,forced air,central,detached garage,"8, 220 sqft",$371 / sqft
4,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,NaN,NaN,NaN,NaN,"10, 019 sqft",NaN


In [11]:
print(data_extension.shape[0])
print(data_extension_schools.shape[0])
print(data.shape[0])

377185
377185
377185


In [12]:
columns = ['Year built','Parking','lotsize']
columns_2 = ['school_rating', 'school_distance']

data_ext = pd.concat([data_extension[columns], data], axis=1,)
data_ext = pd.concat([data_extension_schools[columns_2], data_ext], axis=1, ignore_index=False)

data_ext.head(10)

,school_rating,school_distance,Year built,Parking,lotsize,status,private pool,propertyType,street,baths,...,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,4,2.70,2019,NaN,NaN,active,NaN,single family home,240 heather ln,3.5,...,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,nc,NaN,NaN,NaN,611019,"$418,000"
1,4/10,1.01,2019,NaN,5828 sqft,for sale,NaN,single-family home,12911 e heroy ave,3 baths,...,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 beds,wa,2.0,NaN,NaN,201916904,"$310,000"
2,8/10,1.19,1961,attached garage,"8, 626 sqft",for sale,NaN,single-family home,2005 westridge rd,2 baths,...,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 beds,ca,1.0,NaN,yes,fr19221027,"$2,895,000"
3,9/10,0.10,2006,detached garage,"8, 220 sqft",for sale,NaN,single-family home,4311 livingston ave,8 baths,...,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 beds,tx,3.0,NaN,NaN,14191809,"$2,395,000"
4,5/10,3.03,NaN,NaN,"10, 019 sqft",for sale,NaN,lot / land,1524 kiscoe st,NaN,...,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,fl,NaN,NaN,NaN,861745,"$5,000"
5,0,0.00,1920,NaN,680 sqft,for sale,NaN,townhouse,1624 s newkirk st,NaN,...,"[{'rating': [], 'data': {'Distance': [], 'Grad...",897 sqft,19145,2 beds,pa,2.0,NaN,NaN,paph847006,"$209,000"
6,3,0.80,2006,NaN,"4, 996 sq. ft.",active,NaN,florida,552 casanova ct,NaN,...,"[{'rating': ['3', '3', '1', 'NR'], 'data': {'D...","1,507",34759,NaN,fl,1,NaN,NaN,s5026943,"181,500"
7,2,0.40,1976,NaN,"8, 750 sq. ft.",active,NaN,NaN,6094 mingle dr,NaN,...,"[{'rating': ['4', '2', '2'], 'data': {'Distanc...",NaN,38115,NaN,tn,NaN,NaN,NaN,10063506,"68,000"
8,2,5.60,1970,NaN,124582,active,NaN,single family home,11182 owl ave,2,...,"[{'rating': ['2', '2', '4', '7', '4', 'NR'], '...",3588,50401,3,ia,NaN,NaN,NaN,190988,"$244,900"
9,3/10,0.60,2019,attached garage,"2, 056 sqft",NaN,NaN,single family,8612 cedar plains ln,3,...,"[{'rating': ['4/10', '3/10', '2/10'], 'data': ...","1,930",77080,3,tx,2.0,NaN,NaN,73968331,"$311,995"


Рассмотрим количество и процент пропусков

In [ ]:
data_ext = data_ext.drop(['homeFacts', 'schools'], axis=1)

In [13]:
# Количество пропусков
total = data_ext.isna().sum()
# Процент пропусков
percent = round((total/data_ext.shape[0])*100, 2)

# Объединим в таблицу для лучшего восприятия
missing_data = pd.concat([total,percent],axis=1,keys = ['Total','Percent'])
display(missing_data)

,Total,Percent
school_rating,0,0.00
school_distance,0,0.00
Year built,62374,16.54
Parking,175420,46.51
lotsize,61455,16.29
status,39918,10.58
private pool,373004,98.89
propertyType,34733,9.21
street,2,0.00
baths,106338,28.19


In [14]:
mask = (data_ext['Year built'] == 'no data') | (data_ext['Year built'].isna())
mask_2 = (data_ext['Year built'] == '559990649990') | (data_ext['Year built'] == '1') 

idx = data_ext[mask | mask_2].index
data_ext = data_ext.drop(idx, axis=0)

data_ext['Year built'].unique()

array(['2019', '1961', '2006', '1920', '1976', '1970', '1965', '2015',
       '1996', '1982', '1905', '2008', '1899', '2016', '1954', '1989',
       '2014', '1915', '1994', '1910', '1923', '1960', '1977', '1972',
       '1963', '1981', '1949', '1930', '1991', '1958', '2005', '2017',
       '2018', '1983', '1928', '1998', '1997', '1964', '1974', '1940',
       '1973', '1968', '1999', '1918', '2007', '1984', '1946', '2003',
       '2000', '2004', '1969', '2002', '1952', '1947', '1957', '1978',
       '1956', '1967', '1975', '1988', '1955', '1950', '1959', '2012',
       '1924', '1966', '1992', '2001', '1995', '1985', '2010', '1971',
       '1909', '1953', '1993', '1990', '1980', '1951', '1907', '1885',
       '1941', '1927', '1929', '1901', '1922', '1906', '1945', '1979',
       '2009', '1962', '1913', '2011', '1897', '1987', '2020', '1926',
       '1939', '1925', '2013', '1986', '1943', '1935', '1890', '1900',
       '1895', '1911', '1948', '1942', '1916', '1931', '1944', '1893',
      

Возраст данных примем к данному году - 2023

In [15]:
data_ext['Year built'] = data_ext['Year built'].apply(lambda x: 2023 - int(x))

In [16]:
def to_fill_values(func, features, value=np.NaN):
    def wrap(dataframe, pattern, feature=None):
        df_main = func(dataframe, pattern, feature)
        idx_feature_dict = {}
        for feature in features:
            result = func(dataframe, pattern, feature=feature)
            res_dict_idx = result[1]
            for key, idx in res_dict_idx.items():
                idx_feature_dict[key] = idx_feature_dict.get(key, [])
                idx_feature_dict[key].append(feature)
                df_main.loc[idx, feature] = value
        
        for key_pattern, feature_list in idx_feature_dict.items():
            print(f'Признаки в которых было обнаружено - "{key_pattern}" и заменено на {value}', end=': ')
            print(*feature_list, sep=', ')
        
        return df_main
    
    return wrap


def to_fill_ref_values(func, feature_refer, pattern_refer):
    def wrap(dataframe, pattern, feature):
        result = func(dataframe, pattern, feature)

        df = result[0].copy()
        res_dict_idx = result[1]
        for idx in res_dict_idx.values():
            mask_i = pd.Series(df.index.isin(idx))
            mask_notna = df[feature_refer].notna()
            mask = df[mask_notna][feature_refer].apply(lambda x: True if re.findall(f'{pattern_refer}', x) else False)
            idx = df[mask_notna & mask & mask_i].index
            df.loc[idx, feature] = df.loc[idx, feature_refer]
            
        return df
    
    return wrap


def to_search_values(dataframe, patterns, feature):
    '''
    Args:
        dataframe - целевой датафрейм
        patterns (dict) - словарь паттернов для поиска в признаке датафрейма
        feature - признак в датафреме по которому будет идти поиск
        
    Функция принимает датафрей cловарь паттернов и признак для поиска.
    В цикле идет перебеор вхождения паттерна в строку через маску.
    
    Return: 
        df - Возвращает датафрейм (для дальнейшей передачи в параметры)
        search_idxs_dict - Возврщает словарь с ключами названия паттерна и индексами найдеными по нему
    '''
    if feature == None:
        return dataframe
    
    df = dataframe.copy()
    search_idxs_dict = {}
    # В цикле проходим по мозможным вариантам неявного обозначения пропуска
    if patterns is not None:
        for key, pattern in patterns.items(): 
            mask_notna = df[feature].notna()
            # берем индексы записей с неяным пропуском в столбце из отфильтрованых записей по маске mask_notna
            mask = df[mask_notna][feature].apply(lambda x: True if re.search(f'{pattern}', x) else False)
            idx = df[mask_notna & mask].index
        
            # Если есть записи с неявными пропусками 
            if len(idx) != 0:
                # Выведем признак и уникальные значения с неявным пропуском
                print(f'Найдены значения в признаке : {feature}')
                print(df[mask_notna & mask][feature].value_counts(), '\n')
                search_idxs_dict[key] = idx
            else: continue
        
    else:
        mask = df[feature].isna()
        idx = df[mask].index
        search_idxs_dict['None'] = idx
        
    return (df, search_idxs_dict)


: 

Удалим результаты которые можно считать не подходящими по условию статуса. Нам для модели и для предсказания цены не подходят записи относящиеся к аренде и относящиеся уже к проданным объектам.

> Данные с значение 'sold' и 'for rent', 'closed' и 'lease / purchase pending' в "status" можно считать не подходящими. Так как объекты со статусом "sold" это уже проданные и не могут подходить по формат статуса для рабочей модели регрессии стоимости для продаваемых объектов (то есть для запроса предсказать цену объекта, который уже продан - не имеет основания для запроса). A объекты со статусом "for rent" они не для продажи, а для сдачи в аренду, что не подходит для задачи.

In [104]:
# Паттерн вхождения для строк не приемлемых в данных
patterns = {'rent': r'\brent', 'sold': r'\bsold\b', 'closed': r'\bclosed\b', 'lease / purchase pending': r'lease / purchase pending'}
# Получаем индексы записей с вхождениями
df, res_dict = to_search_values(data_ext, patterns, 'status')
# В цилке проходим по наборам индексов по каждому паттерну
# берем индексы вхождения найденые по паттерну и удаляем соотвествующие им записи
for idx in res_dict.values():
    data_ext = data_ext.drop(idx, axis=0)

Найдены значения в признаке : status
for rent              398
apartment for rent      7
condo for rent          7
Name: status, dtype: int64 

Найдены значения в признаке : status
recently sold      287
sold: $285, 000      1
sold: $15, 000       1
Name: status, dtype: int64 

Найдены значения в признаке : status
closed    18
Name: status, dtype: int64 

Найдены значения в признаке : status
lease / purchase pending    3
Name: status, dtype: int64 



In [ ]:
mask = (data_ext['Year bild'].isna()) & (data_ext['ProprtyType'].isna())\
    & (data_ext['sqft'].isna()) & (data_ext['beds'].isna())
idx = data_ext
data_ext = data_ext.drop[idx, axis=0]

Посмотрим на наличие неявных обозначений пропусков и приведем неявные пропуски к явному виду

In [105]:
# Cписок с возможными неявными пропусками
implicit_missings_patterns = {
    'unknown': r'\bunknown\b',
    'n / a': r'\bn / a\b',
    'na': r'\bna\b',
    'nan': r'\bnan\b',
    'none': r'\bnone\b',
    'n / k': r"\bn / k\b",
    'undisclosed': r'\bundisclosed\b',
    'not disclosed': r'\bnot disclosed\b',
    'not available': r'\bnot available\b',
    'none': r'\bnone\b',
    'not data': r'\bnot\sdata\b',
    'not info': r'\bnot\sinfo\b'
}

features = data_ext.columns
# Определим функцию приведения 
fill_implicit = to_fill_values(to_search_values, features)
data = fill_implicit(data_ext, implicit_missings_patterns)

Найдены значения в признаке : propertyType
unknown    30
Name: propertyType, dtype: int64 

Найдены значения в признаке : street
unknown address         72
unknown                  2
unknown rd #lot port     1
Name: street, dtype: int64 

Найдены значения в признаке : street
99999 emma rd #n / a                           1
12060 old statesville rd unit n / a            1
5647 vineyard ln #n / a                        1
9563 gainey ranch ave #n / a                   1
6164 reitzel dr #n / a                         1
5266 sw 123rd ave #n / a                       1
n / a ruben torres blvd                        1
10878 sw 89th st #n / a                        1
n / a harbor island dr                         1
n / a hawkins & bill owens pkwy                1
1038 e sevier ave #n / a                       1
n / a beth page ct                             1
10878 sw 89th st unit n / a                    1
9 street lot n a #lot n / a                    1
n / a n central ave                   

Также были замечены значения пропусков которые имеют обозначение '--' и '~'

In [106]:
# Cписок с возможными неявными пропусками
implicit_missings_patterns = {'--': r'--', '~': r'^~$'}

features = data_ext.columns
for feature in features:
    to_search_values(data_ext, implicit_missings_patterns, feature)

Найдены значения в признаке : street
4822 w avenida del rey --            1
6551 w gold mountain pass -- #28     1
7051 w villa chula --                1
1025 wood owl way lot 26--jackson    1
10308 w calle del oro --             1
-- glamis cir                        1
9923 w calle encorvada --            1
Name: street, dtype: int64 

Найдены значения в признаке : baths
-- baths    509
Name: baths, dtype: int64 

Найдены значения в признаке : baths
~    782
Name: baths, dtype: int64 

Найдены значения в признаке : city
--    9
Name: city, dtype: int64 

Найдены значения в признаке : sqft
--         535
-- sqft    285
Name: sqft, dtype: int64 

Найдены значения в признаке : zipcode
--    2
Name: zipcode, dtype: int64 

Найдены значения в признаке : beds
-- bd      1012
-- sqft      97
Name: beds, dtype: int64 



Повторно посмотрим на пропуски 

In [107]:
# Количество пропусков
total = data_ext.isna().sum()
# Процент пропусков
percent = round((total/data_ext.shape[0])*100, 2)

# Объединим в таблицу для лучшего восприятия
missing_data = pd.concat([total,percent],axis=1,keys = ['Total','Percent'])
display(missing_data)

,Total,Percent
status,39918,10.60
private pool,372283,98.89
propertyType,34752,9.23
street,1891,0.50
baths,106115,28.19
homeFacts,0,0.00
fireplace,273618,72.68
city,34,0.01
schools,0,0.00
sqft,40538,10.77


> Как видим были обнаружены неявные пропуски и заменены на явные

> Неявные пропуски обозначеные '--' и '~' пока не заменены, поскольку есть предположение, что по ним как по указателям можно найти соотвествующие значения в других признаках, так как в некоторых признаках наблюдаются такие значения подходящие под другие признаки.

Помотрим на записи с пропусками в "target"

In [108]:
mask_isna = data_ext['target'].isna()
print('Значения в признаках при пропуске в "target":', '\n')
print(data_ext[mask_isna]['status'].value_counts(), '\n')
print(data_ext[mask_isna]['propertyType'].value_counts(), '\n')
print(data_ext[mask_isna]['sqft'].value_counts(), '\n')
print(data_ext[mask_isna]['beds'].value_counts(), '\n')
print(data_ext[mask_isna]['baths'].value_counts(), '\n')
print(data_ext[mask_isna]['MlsId'].value_counts(), '\n')
print(data_ext[mask_isna]['mls-id'].value_counts(), '\n')

Значения в признаках при пропуске в "target": 

foreclosure                  749
pre-foreclosure / auction    279
foreclosed                   165
/ auction                    137
pre-foreclosure              119
auction                       43
new construction              16
pending                        5
for sale                       4
active                         1
Name: status, dtype: int64 

single family            979
single-family home       647
apartment                124
condo                    107
singlefamilyresidence     41
townhouse                 29
multi family              22
multiple occupancy        13
mobile / manufactured     12
lot / land                 9
multi-family               8
miscellaneous              8
single family home         3
cooperative                2
other                      1
colonial                   1
mfd / mobile home          1
coop                       1
land                       1
Name: propertyType, dtype: int64 

-- sqft

> Так как для данных с пропусками "target" имеется разные значения данных и некоторая условность в типе и статусе объекта (foreclosuer и single famile (по количеству)), то данные записи с пропусками в "target" рассмотрим либо на удаление либо на заполнение медианным значением для характерных типа и статуса (foreclosuer и single famile).

In [109]:
# Резервная копия
data_copy = data_ext.copy()

### Cодержание признаков и их уникальные значения 

#### Первоначально рассмотрим 'private pool' и 'PrivatePool'

In [110]:
print('Уникальные значения признака "private pool"', data_ext['private pool'].unique(), sep='\n', end='\n\n')
print('Уникальные значения признака "PrivatePool"', data_ext['PrivatePool'].unique(), sep='\n')

Уникальные значения признака "private pool"
[nan 'yes']

Уникальные значения признака "PrivatePool"
[nan 'yes']


> Видно, что "PrivatePool" имеет уникальные значения такие же как и "private pool".

> Так как наблюдается то, что в значениях имеется только 'yes' о наличие бассейна, то все прпуски можно считать как значение 'no'.

In [111]:
# определим различия значений в двух признаках
compare = data_ext['PrivatePool'].compare(data_ext['private pool'], keep_equal=False)

# Выведем различающиеся строки для признаков
display(
    'Таблица рзаличных значений между признаками "PrivatePool", "private pool"',
    compare)
# Выведем различающиеся строки для признаков для ненулевых значений
display(
    'Таблица рзаличных значений между признаками "PrivatePool", "private pool" для непустых значений',
    compare[(compare['other'].notna()) & (compare['self'].notna())])

'Таблица рзаличных значений между признаками "PrivatePool", "private pool"'

,self,other
2,yes,NaN
12,yes,NaN
19,yes,NaN
43,yes,NaN
60,yes,NaN
...,...,...
377168,yes,NaN
377170,yes,NaN
377171,yes,NaN
377174,yes,NaN


'Таблица рзаличных значений между признаками "PrivatePool", "private pool" для непустых значений'

,self,other


> Нет различающихся значений для непустых значений одновременно.

> Значения в признаках - определяют только указание на наличие бассейна (yes)

Обработаем признаки 'private pool' и 'PrivatePool'

Определим функцию которая, приведет к нижнему регистру, объединим и заполним пропуски значением 'no'. Внесем изменения в один из признаков, другой удалим.

In [112]:
# Вносим объединенные значения в PrivatePool
# добавляем к PrivatePool, где он пустой, не пустые значения private pool 
# где пусто определяем как 0, а остальное как 1

data_ext['PrivatePool'] = data_ext['PrivatePool']\
    .combine_first(data_ext['private pool'])\
    .apply(lambda x: 0 if x is np.NaN else 1)

data_ext = data_ext.drop('private pool', axis=1)
data_ext.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 376463 entries, 0 to 377184
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        336545 non-null  object
 1   propertyType  341711 non-null  object
 2   street        374572 non-null  object
 3   baths         270348 non-null  object
 4   homeFacts     376463 non-null  object
 5   fireplace     102845 non-null  object
 6   city          376429 non-null  object
 7   schools       376463 non-null  object
 8   sqft          335925 non-null  object
 9   zipcode       376463 non-null  object
 10  beds          285308 non-null  object
 11  state         376463 non-null  object
 12  stories       226256 non-null  object
 13  mls-id        24941 non-null   object
 14  PrivatePool   376463 non-null  int64 
 15  MlsId         310024 non-null  object
 16  target        374284 non-null  object
dtypes: int64(1), object(16)
memory usage: 59.8+ MB


#### Рассмотрим признаки 'mls-id' и 'MlsId'

Посморим какие значения встрачаются максимально по количеству (имеются ли такие и что это за значения если есть)

In [113]:
# В цикле выведем количество значений для признаков
for feature in ['MlsId', 'mls-id']:
    print(f'Максимальные по количеству значения встречающиехся в "{feature}"')
    print(data_ext[feature].value_counts()[:5], end='\n'*2)

    print('Уникальных значений: {}'.format(data_ext[feature].nunique()), end='\n'*2)

Максимальные по количеству значения встречающиехся в "MlsId"
no mls                       40
no mls #                     17
a, houston, tx 77008         13
12a, orlando, fl 32833       11
1, south boston, ma 02127     9
Name: MlsId, dtype: int64

Уникальных значений: 232646

Максимальные по количеству значения встречающиехся в "mls-id"
no          3
no mls#     3
983469      2
20193733    2
1530716     2
Name: mls-id, dtype: int64

Уникальных значений: 24905



Посмотрим какие записи о сделках представлены в датафрейме в ключе значений "No MLS"

In [114]:
display(data_ext[data_ext['MlsId']=='no mls'][['status', 'propertyType']])
display(data_ext[data_ext['MlsId']=='no mls #'][['status', 'propertyType']])

,status,propertyType
8196,foreclosure,single-family home
9028,foreclosure,single-family home
9321,foreclosure,single-family home
16282,foreclosure,single-family home
44742,for sale,lot / land
63946,foreclosure,condo
65330,NaN,condo
72013,foreclosure,condo
99464,NaN,single family
104654,NaN,single family


,status,propertyType
9680,foreclosure,single-family home
53383,foreclosure,single-family home
73724,foreclosure,single-family home
82514,foreclosure,single-family home
100756,foreclosure,multi-family
129853,foreclosure,single-family home
132986,foreclosure,single-family home
135258,foreclosure,single-family home
135535,foreclosure,single-family home
207419,foreclosure,single-family home


In [115]:
display(data_ext[data_ext['mls-id']=='no mls#'][['status', 'propertyType']])
display(data_ext[data_ext['mls-id']=='no'][['status', 'propertyType']])

,status,propertyType
109388,foreclosure,single family
168717,foreclosure,single family
328744,foreclosure,single family


,status,propertyType
21525,foreclosure,single family
25936,foreclosure,single family
109576,foreclosure,single family


> Можем наблюдать то, что для записей, которые не отмечены что нет их индетификатора MLS характрено, что это тип объекта "single-family home", a статус "foreclosure". 
<br/><br/>Это можно объяснить тем, что такой статус "foreclosure" («лишён права выкупа») определяет объект вернувшийся кредитору и продажа происходит по отдельному сценарию или ее нет (так как кредитор может оставить без продажи вернувшееся имущество). То есть записи в базах по MLS не внесены (нет индетефикатора MLS) либо удалены. и как правило это дома обучные дома по типу single-family home взятые в ипотеку.<br/><br/>Таким образом можем считать, что можно наличие пропусков в propertyType для объектов со status = foreclosure принять как single-family home.

Проверим соотношение статуса 'foreclosure' и типа объекта

In [116]:
# Маска непустых значений
mask_notna = data_ext['propertyType'].notna()
# Маска по условию
mask = data_ext[mask_notna]['propertyType'].apply(lambda x: True if ('single' in x) else False)

print('Значения в "status" по количеству для значения "propertyType" = single:')
print(data_ext[mask_notna & mask]['status'].value_counts(), '\n')
print('Кол-во записей для значения "propertyType" = single: {}'.format(mask.sum()))
print('Кол-во пропусков в "status" для значения "propertyType" = single: {}'.format(data_ext[mask_notna & mask]['status'].isna().sum()))

Значения в "status" по количеству для значения "propertyType" = single:
for sale                     117003
active                        26748
foreclosure                    5535
new construction               4034
pending                        3364
                              ...  
coming soon: nov 26.              1
pending w / escape clause         1
coming soon: dec 25.              1
coming soon: nov 11.              1
coming soon: dec 23.              1
Name: status, Length: 111, dtype: int64 

Кол-во записей для значения "propertyType" = single: 190042
Кол-во пропусков в "status" для значения "propertyType" = single: 23401


In [117]:
# Маска непустых значений
mask_notna = data_ext['status'].notna()
# Маска по условию
mask = data_ext[mask_notna]['status'].apply(lambda x: True if ('foreclos' in x) and ('pre' not in x) else False)

print('Значения в "propertyType" по количеству для значения "status" ~ foreclosure:')
print(data_ext[mask_notna & mask]['propertyType'].value_counts(), '\n')
print('Кол-во записей для значения "status" ~ foreclosure: {}'.format(mask.sum()))
print('Кол-во пропусков в "propertyType" для значения "status" ~ foreclosure: {}'.format(data_ext[mask_notna & mask]['propertyType'].isna().sum()))

Значения в "propertyType" по количеству для значения "status" ~ foreclosure:
single-family home       5261
single family             632
condo                     578
multi-family              235
townhouse                 165
lot / land                153
apartment                  49
mobile / manufactured      44
singlefamilyresidence      39
coop                       24
multi family               20
multiple occupancy         16
single family home          2
cooperative                 1
Name: propertyType, dtype: int64 

Кол-во записей для значения "status" ~ foreclosure: 7230
Кол-во пропусков в "propertyType" для значения "status" ~ foreclosure: 11


> Видно, что можем утвердить, что большинство объектов со статусом 'foreclosure' имеет тип 'single-family home'

> Признаки 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система
мультилистинга). Так как признаки 'mls-id' и 'MlsId' являются признаками для идентификации объекта в общих базах риэлторов, то мы можем их удалить, так как для отдельной индетефикации у нас есть zipcode и street (адресс объекта).
<br/><br/>
Признаки 'mls-id' и 'MlsId' могли бы быть нужны если по ним можно было бы погрузить и соединить данную таблицу с какой-либо другой с дополнительной информацией - используя эти признаки как ключ объединения.

In [96]:
mask_1 = data_ext['MlsId']=='no mls'
mask_2 = data_ext['MlsId']=='no mls #'
idx = data_ext[mask_1 | mask_2].index
data_ext.loc[idx, 'propertyType'] = 'foreclosure'

data_ext = data_ext.drop(['mls-id' , 'MlsId'], axis=1)

data_ext.info()


KeyboardInterrupt



#### Рассмотрим признак status

In [60]:
print('Максимальные по количеству (топ-10) значения встречающиехся в "status"')
print(data_ext['status'].value_counts()[:10], end='\n'*2)

print('Уникальных значений: {}'.format(data_ext['status'].nunique()), end='\n'*2)

display('Уникальные значения:',data_ext['status'].unique())
print('-'*40)

display('Топ значение по количеству (прценту от пропусков) для записей с пропуском в "status"')
for column in data_ext.columns:
    display(data_ext[data_ext['status'].isna()][column].value_counts()[:5]/data_ext['status'].isna().sum()*100)

Максимальные по количеству (топ-10) значения встречающиехся в "status"
for sale                     199571
active                       105207
foreclosure                    6769
new construction               5475
pending                        4807
pre-foreclosure                2119
pre-foreclosure / auction      1560
p                              1488
under contract show            1183
/ auction                       936
Name: status, dtype: int64

Уникальных значений: 143



'Уникальные значения:'

array(['active', 'for sale', nan, 'new construction', 'new', 'pending',
       'p', 'active / contingent', 'pre-foreclosure / auction',
       '/ auction', 'under contract', 'under contract showing',
       'pre-foreclosure', 'under contract backups', 'foreclosure',
       'active under contract', 'foreclosed', 'option pending',
       'under contract show', 'auction', 'a active', 'contingent',
       'pending continue to show', 'price change', 'back on market',
       'active option', 'coming soon: nov 21.',
       'contingent finance and inspection', 'coming soon: dec 4.',
       'p pending sale', 'coming soon: nov 23.',
       'active with contingencies', 'pending ab', 'pf', 'contingent show',
       'contract p', 'contingent take backup', 'backup contract',
       'option contract', 'pending inspection', 'active option contract',
       'c', 'auction - active', 'pi', 'due diligence period',
       'contract contingent on buyer sale', 'u under contract',
       'coming soon: nov 29.

----------------------------------------


'Топ значение по количеству (прценту от пропусков) для записей с пропуском в "status"'

...

3    34.643519
4    21.684453
2    18.625683
1     6.418157
5     6.067438
Name: beds, dtype: float64

fl    55.741771
tx    40.270054
ca     1.367804
wa     0.841726
nc     0.210431
Name: state, dtype: float64

1.0    25.903101
2.0    19.041535
1       9.825142
2       5.295857
0.0     4.531790
Name: stories, dtype: float64

0    81.567213
1    18.432787
Name: PrivatePool, dtype: float64

$225,000    0.473471
$350,000    0.470965
$399,000    0.435894
$299,000    0.423368
$175,000    0.393306
Name: target, dtype: float64

> Наиболее большие по количеству значения for sale и For sale и active. По идеи эти два статуса можно счить одним и темже, for sale (для продажи) - объект готовый на продажу, а active - статус объекта, который находится в продаже, но не в ожидании по сделке, то есть просматирвается и прочее. Таким образом будем их считать одним и темже значением в дальнейшем.

> Наблюдается большое множество уникальных значений при этом можно заметить, что среди них имеются такие как: <br/>
    1. Записи в одного и тогоже в разном регистре и форме
<br/>
    2. Смешанные записи через "/".
<br/>
    3. Статусы с указание "sold" (продано) и "for rent" (для аренды)
<br/>
    4. Ряд статусов которые могут быть сгруппированными.

> Для пропушенных значений в "status" характерны:<br/>
<br/>- значения propertyType c вхождением 'single' (58.6%)
<br/>- значения state - fl и tx (55.7% и 40.3% соответсвенно)

Посмотрим какие значения в "status" имеются для характерных значений при пропуске

In [27]:
mask_notna_1 = data['propertyType'].notna()
mask_1 = data[mask_notna_1]['propertyType'].apply(lambda x: True if 'single' in x else False)

mask_notna_3 = data['state'].notna()
mask_3 = data[mask_notna_3]['state'].apply(lambda x: True if ('fl' in x) or ('tx' in x) else False)

data[(mask_notna_1 & mask_notna_3) & (mask_1 & mask_3)]['status'].value_counts()[:10]

for sale                     58882
active                       11271
pending                       1826
foreclosure                   1727
new construction              1704
pre-foreclosure                809
pre-foreclosure / auction      462
foreclosed                     172
auction                        150
pending continue to show       147
Name: status, dtype: int64

> Для характерных значений в других признаках при пропусках в "status" для не пустых значений "status" преобладает 'for sale' и 'active' - эти значения нами рассматриваются как одно и тоже

#### Рассмотрим признак propertyType

In [80]:
print('Максимальные по количеству (топ-10) значения встречающиехся в "propertyType"')
print(data_ext['propertyType'].value_counts()[:10], end='\n'*2)

print('Уникальных значений: {}'.format(data_ext['propertyType'].nunique()), end='\n'*2)

display('Уникальные значения:', data_ext['propertyType'].unique())
print('-'*40)

display('Топ значение по количеству (прценту от пропусков) для записей с пропуском в "propertyType"')
for column in data_ext.columns:
    display(data_ext[data_ext['propertyType'].isna()][column].value_counts()[:5]/data_ext['propertyType'].isna().sum()*100)

Максимальные по количеству (топ-10) значения встречающиехся в "propertyType"
single-family home                     91806
single family                          62867
condo                                  42472
single family home                     31728
lot / land                             20541
townhouse                              18334
land                                   10934
multi-family                            7771
condo / townhome / row home / co-op     7701
traditional                             5912
Name: propertyType, dtype: int64

Уникальных значений: 1269



'Уникальные значения:'

array(['single family home', 'single-family home', 'lot / land', ...,
       'cabin / lodge, contemporary, converted barn, cottage, loft with bedrooms, log home, post & beam',
       'lake house, single detached, tudor',
       'bilevel, converted dwelling, loft with bedrooms, condo / unit'],
      dtype=object)

----------------------------------------


'Топ значение по количеству (прценту от пропусков) для записей с пропуском в "propertyType"'

...

one    8.172192
2      3.352325
two    3.323550
1      2.051680
3      0.667587
Name: stories, dtype: float64

408807        0.002878
297323        0.002878
r1236023      0.002878
iv19147238    0.002878
869412        0.002878
Name: mls-id, dtype: float64

yes    0.01151
Name: PrivatePool, dtype: float64

12a, orlando, fl 32833    0.031653
11a, orlando, fl 32833    0.023020
3a, orlando, fl 32833     0.011510
14230543                  0.008633
1367153                   0.008633
Name: MlsId, dtype: float64

350,000    0.466160
225,000    0.448895
250,000    0.425875
275,000    0.408610
325,000    0.399977
Name: target, dtype: float64

In [ ]:
idx = data_ext[data_ext['propertyType'] == 'land'].index
for column in data_ext.columns:
    display(data_ext.loc[idx, column].value_counts()[:10])

> Сильно по количеству выделяется статус single family и его "аналоги"

> Также как и в признаке 'status' наблюдается большое множество уникальных значений, множество значений одного и тогоже в различной форме записи. Также наблюдаются смешанные значения (через '/') и значения которые могут быть сгрупированны.

> Для пропушенных значений в "propertyType" характерны:<br/>
<br/>- значения propertyType c вхождением 'single' (88.6%)
<br/>- значения state - fl и tx и ny (42.1% и 18.8% и 16.9% соответсвенно)

Посмотрим какие значения в "status" имеются для характерных значений при пропуске

In [78]:
result = to_search_values(data_ext, {'acitve': r'(?:^active|^a\s)', 'for sale': r'\bfor sale\b'}, 'status')
idx_1=[]
for el in result[1].values():
    idx_1.extend(el) 

mask = (data_ext['state'] == 'ny') | (data_ext['state'] == 'fl') | (data_ext['state'] == 'tx')
idx_2 = data_ext[mask].index

idx = set(idx_1).intersection(idx_2)

data_ext.loc[idx, 'propertyType'].value_counts()[:10]

Найдены значения в признаке : status
active                       105207
active under contract           718
a active                        443
active with offer                49
active backup                    39
active / contingent              15
active with contingencies        13
active - auction                 10
active contingency                8
active option                     7
active option contract            7
active contingent                 6
active - contingent               2
active with contract              2
Name: status, dtype: int64 

Найдены значения в признаке : status
for sale    199571
Name: status, dtype: int64 



C:\Users\User\AppData\Local\Temp\ipykernel_4936\3344351910.py:11: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  data.loc[idx, 'propertyType'].value_counts()[:10]


single-family home    50704
condo                 17428
single family         12984
lot / land            11755
single family home     8457
townhouse              6208
land                   5980
traditional            4586
multi-family           4082
coop                   3200
Name: propertyType, dtype: int64

> Для характерных значений в других признаках при пропусках в "propertyType" для не пустых значений "propertyType" преобладает 'single family' и его вариации.

#### Рассмотрим признак "street"

In [83]:
print('Максимальные по количеству значения встречающиехся в "street"')
print(data_ext['street'].value_counts()[:10], end='\n'*2)

display('Уникальные значения:', data_ext['street'].unique())

Максимальные по количеству значения встречающиехся в "street"
address not disclosed    672
undisclosed address      517
(undisclosed address)    391
address not available    175
2103 e state hwy 21       57
11305 gulf fwy            54
17030 youngblood rd.      38
ne 58th cir               34
9470 lancaster rd. sw     32
1 palmer dr               27
Name: street, dtype: int64



'Уникальные значения:'

array(['240 heather ln', '12911 e heroy ave', '2005 westridge rd', ...,
       '3530 n lake shore dr #4b', '15509 linden blvd', '7810 pereida st'],
      dtype=object)

> Видно, что присутсвуют значения, которые являются указателями к тому, что адресс не известен или скрыт.

> Слишком много уникальных значений. Количество уникальных значений соразмерно с количеством записей в датафреме. 

> Возможно, можно было бы по данному признаку собрать информацию через парсинг в интернете как например характеристики района, но так как есть пропуски и возможны неточные значения в данном признаке, это не рассматривается

Посмотрим какие значения имеются с вхождением 'Address' или 'closed'

In [562]:
# Маска не пустых значений для "street"
mask_notna = data_ext['street'].notna()
# Маска наличия 'Address' в признаке "street"
mask = data_ext[mask_notna]['street'].apply(lambda x: True if ('Address' in x) or ('closed' in x) else False)

data_ext[mask_notna & mask]['street'].unique()

array(['Address Not Disclosed', '5580 Clarke Address Dr #54 ',
       'Undisclosed Address', 'Address Not Available',
       '(undisclosed Address)', 'Unknown Address', 'Address Pending',
       '5580 Clarke Address # 54', 'Address Unassigned Burnt Store Rd ',
       'Undisclosed Rd NE', '41619 Re Undisclosed',
       '112016 Rs Undisclosed', 'Address Withheld By Seller Las Vegas',
       'N Address #20', '61819 Undisclosed', 'N Street Address #29',
       '5580 Clarke Address Dr #54'], dtype=object)

> Видим такие значения как Address Not Disclosed, Undisclosed Address, (undisclosed Address), Address Not Available, Unknown Addressб, Address Pending, Undisclosed и прочее

Подитожим.

> Таким образом данных признак подлежит удалению, так как является неинформативным с больши количеством уникальных значений, и достаточно большим количеством пропусков, для такого типа признака.

#### Рассмотрим признак "baths"

In [84]:
print('Максимальные по количеству (топ-10) значения встречающиехся в "baths"')
print(data_ext['baths'].value_counts()[:10], end='\n'*2)

print('Уникальных значений: {}'.format(data_ext['baths'].nunique()), end='\n'*2)

display('Уникальные значения:', data_ext['baths'].unique())
print('-'*40)

display('Топ значение по количеству (прценту от пропусков) для записей с пропуском в "baths"')
for column in data_ext.columns:
    display(data_ext[data_ext['baths'].isna()][column].value_counts()[:5]/data_ext['baths'].isna().sum()*100)

Максимальные по количеству (топ-10) значения встречающиехся в "baths"
2 baths         52221
3 baths         35443
2               20451
2.0             16576
4 baths         14742
3.0             10869
3               10112
bathrooms: 2     9535
2.5              8113
bathrooms: 3     6613
Name: baths, dtype: int64

Уникальных значений: 226



'Уникальные значения:'

array(['3.5', '3 baths', '2 baths', '8 baths', nan, '2', '3',
       'bathrooms: 2', '1,750', '4 baths', '2 ba', 'bathrooms: 5',
       '1,000', '7 baths', '2.0', '3.0', 'bathrooms: 1', '4.0',
       '2.1 baths', '2.5 baths', '1', 'bathrooms: 3', '4.5', '6 baths',
       'bathrooms: 4', '3 ba', '5', '2,500', '5.5 baths', '1.0',
       '5 baths', '1.5', '4', '~', '2.5', '4,000', '3.5 baths', '2,000',
       '3,000', '8.0', '1 ba', '3.5 ba', '0', '5.0', '1,500', '7.0',
       '1,250', '9 baths', '2,250', '6.0', '12 baths', '5.5', '3,500',
       '1.5 baths', '2,750', 'bathrooms: 6', '4.5 baths', '750', '5.5+',
       '6', '10 baths', '6 ba', 'bathrooms: 19', '10.0', '1.5 ba', '4 ba',
       '12 ba', '2.5+', '8', '7.5+', 'bathrooms: 10', '0 / 0', 'sq. ft.',
       '5 ba', '4.5+', '18 baths', '-- baths', 'bathrooms: 7', '7', '18',
       '3.5+', '1.5+', '11 baths', '5,000', '1.75 baths', '9', '12.0',
       '1.1 baths', '6.5', 'bathrooms: 8', '10', '19 baths',
       'bathrooms: 9', '16 ba

----------------------------------------


'Топ значение по количеству (прценту от пропусков) для записей с пропуском в "baths"'

...

1.0    8.639683
one    3.972106
1      3.226688
2.0    3.139990
2      2.932667
Name: stories, dtype: float64

no mls#    0.001885
no mls     0.001885
3103568    0.000942
533964     0.000942
3008023    0.000942
Name: mls-id, dtype: float64

yes    2.64807
Name: PrivatePool, dtype: float64

12a, orlando, fl 32833    0.010366
11a, orlando, fl 32833    0.007539
no mls #                  0.004712
1367153                   0.004712
14168512                  0.004712
Name: MlsId, dtype: float64

$1,000      0.604062
$35,000     0.487207
$25,000     0.479668
$45,000     0.421241
$150,000    0.381661
Name: target, dtype: float64

> Записи представлены в разных формах.
Есть записи через '/'. Есть опечатка значение 241

> Анализ информации из интернета показал, что встречаются ванные по своей структуре - укомплектованности являются не целыми и обозначаются как половинные или на 1 или на 3 четверти. Такие значения мы можем и наблюдать.

> Наибольшие по количеству значения это 2 или 3 ванные. 

In [88]:
display('Топ значение по количеству (прценту от пропусков) для записей с пропуском в "baths"')

implicit_missings_patterns = {'--': r'--'}

result = to_search_values(data_ext, implicit_missings_patterns, 'baths')
idx = []
for el in result[1].values():
    idx.extend(el) 

for column in data_ext.columns:
    display(data_ext.loc[idx, column].value_counts()[:5]/data_ext.loc[idx].shape[0]*100)

'Топ значение по количеству (прценту от пропусков) для записей с пропуском в "baths"'

Найдены значения в признаке : baths
-- baths    509
Name: baths, dtype: int64 



for sale                79.371316
auction                 12.180747
foreclosure              4.322200
new construction         3.929273
coming soon: dec 12.     0.196464
Name: status, dtype: float64

yes    2.750491
Name: private pool, dtype: float64

multi family             54.420432
single family            32.809430
singlefamilyresidence     6.483301
apartment                 2.161100
condo                     2.161100
Name: propertyType, dtype: float64

...

2.0    26.129666
1.0    22.593320
0.0    19.253438
3.0     4.715128
4.0     0.982318
Name: stories, dtype: float64

32405       0.196464
16400830    0.196464
6046569     0.196464
40880520    0.196464
727518      0.196464
Name: mls-id, dtype: float64

Series([], Name: PrivatePool, dtype: float64)

Series([], Name: MlsId, dtype: float64)

$24,900     1.571709
$249,900    0.982318
$10,000     0.785855
$114,900    0.785855
$99,900     0.785855
Name: target, dtype: float64

Посмотрим какие объекты имеют значения "baths" с вхождением '/'

In [564]:
mask_notna = data_ext['baths'].notna()
mask = data_ext[mask_notna]['baths'].apply(lambda x: True if '~' in x else False)
display(data_ext[mask_notna & mask])
display(data_ext[mask_notna & mask][['propertyType', 'status']]['propertyType'].value_counts())
display(data_ext[mask_notna & mask][['propertyType', 'status']]['status'].value_counts())

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
154,Active,NaN,"Tri-Level, Northwestern Contemporary",2587 Magnolia Blvd W,~,"{'atAGlanceFacts': [{'factValue': '1967', 'fac...",2,Seattle,"[{'rating': ['8', '8', '6', '8', '9', '6'], 'd...","3,190",98199,Baths,WA,NaN,NaN,NaN,1497930,"2,595,000"
889,Active,NaN,NaN,9825 E 138th Pl,~,"{'atAGlanceFacts': [{'factValue': '2011', 'fac...",3,BRIGHTON,"[{'rating': ['7', '7', '2', 'NR', '3'], 'data'...","9,680",80602,Baths,CO,NaN,NaN,NaN,3401038,"2,100,000"
980,Active,NaN,Townhouse,7116 27th Ave SW,~,"{'atAGlanceFacts': [{'factValue': '2010', 'fac...",1,Seattle,"[{'rating': ['3', '5', '6', '4'], 'data': {'Di...","1,720",98106,Baths,WA,NaN,NaN,NaN,1546133,"519,000"
1377,Active,NaN,"Multi-Level, Colonial",1600 35th Ave,~,"{'atAGlanceFacts': [{'factValue': '1901', 'fac...",2,Seattle,"[{'rating': ['5', '6'], 'data': {'Distance': [...","5,630",98122,Baths,WA,NaN,NaN,NaN,1534082,"2,995,000"
1740,Active,NaN,Traditional,421 Rosemont Dr,~,"{'atAGlanceFacts': [{'factValue': '2004', 'fac...",3,Durham,"[{'rating': ['5', 'NR', '10', '3', '5'], 'data...","7,163",27713,Baths,NC,NaN,NaN,NaN,2243604,"2,500,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375334,Active,NaN,"1 Story, Contemporary",203 Elizabeth Rd,~,"{'atAGlanceFacts': [{'factValue': '2015', 'fac...",1,San Antonio,"[{'rating': ['6', '7', '6'], 'data': {'Distanc...","5,135",78209,Baths,TX,NaN,NaN,NaN,1425556,"1,350,000"
375480,Active,NaN,Single Detached,5414 Edlen Dr,~,"{'atAGlanceFacts': [{'factValue': '1952', 'fac...",1,Dallas,"[{'rating': ['7', '3', '4', '10'], 'data': {'D...","4,237",75220,Baths,TX,NaN,NaN,NaN,14191136,"2,225,000"
375704,Active,NaN,Traditional,7521 Briar Rose Dr,~,"{'atAGlanceFacts': [{'factValue': '2000', 'fac...",1,Houston,"[{'rating': ['7', '5', '1'], 'data': {'Distanc...","5,592",77063,Baths,TX,2,NaN,NaN,47354336,"1,498,000"
376297,Active,NaN,Single Detached,6706 Lakehurst Ave,~,"{'atAGlanceFacts': [{'factValue': '2018', 'fac...",1,Dallas,"[{'rating': ['8', '3', '4', '10'], 'data': {'D...","6,219",75230,Baths,TX,NaN,NaN,NaN,14237467,"2,350,000"


Traditional                                                  77
Contemporary                                                 40
Urban Contemporary                                           38
Colonial                                                     35
Single Detached, Traditional                                 34
                                                             ..
Single Detached, Early American, Traditional                  1
1 Story with Basement, Modern                                 1
Contemporary, Florida                                         1
Single Detached, Contemporary/Modern, Other (See Remarks)     1
Two Story                                                     1
Name: propertyType, Length: 157, dtype: int64

Active                770
Pending                 3
Price Change            3
Pending Inspection      2
New                     2
Name: status, dtype: int64

#### Рассмотрим признак beds

In [565]:
print('Максимальные по количеству значения встречающиехся в "beds"')
print(data_ext['beds'].value_counts()[:10], end='\n'*2)

print('Уникальных значений: {}'.format(data_ext['beds'].nunique()), end='\n'*2)

display('Уникальные значения:', data_ext['beds'].unique())

Максимальные по количеству значения встречающиехся в "beds"
3 Beds    53459
4 Beds    35418
3         31406
2 Beds    26362
4         20030
2         16110
Baths     15282
3 bd      12877
5 Beds    11271
4 bd       8265
Name: beds, dtype: int64

Уникальных значений: 1184



'Уникальные значения:'

array(['4', '3 Beds', '5 Beds', ..., '9,565 sqft', '1.74 acres',
       '8,479 sqft'], dtype=object)

> Наиболее представленные значения 3 и 4 для указателя beds

> Есть записи с значением площади (например '9,565 sqft' или '1.74 acres'). В дальнейшем скорее всего такие записи нужно будет удалить

Рассмотрим записи со значениями данного признака, в которых есть 'sqft' или 'acres'

In [627]:
mask_notna = data_ext['beds'].notna()
mask = data_ext[mask_notna]['beds'].apply(lambda x: True if ('sqft' in x) or ('acres' in x) else False)
df_m = data_ext[mask_notna & mask]

print(f'Кол-во записей со значениями площади в "beds" {mask.sum()}', '\n')

print(df_m['status'].value_counts())
print(
    'Пропусков в "status" при значениях площади в "beds": {}'.format(df_m['status'].isna().sum()),
    '\n')

print(df_m['propertyType'].value_counts())
print(
    'Пропусков в "propertyType" при значениях площади в "beds": {}'.format(df_m['propertyType'].isna().sum()), 
    '\n')

print(df_m['sqft'].value_counts())
print(
    'Пропусков в "sqft" при значениях площади в "beds": {}'.format(df_m['sqft'].isna().sum()),
    '\n')

print('Значения "beds" по количеству для одновременных пропусков в "status" и "propertyType"')
print(data_ext[data_ext['status'].isna() & data_ext['propertyType'].isna()]['beds'].value_counts())

Кол-во записей со значениями площади в "beds" 3027 

Foreclosure                  28
Foreclosed                   22
Pre-foreclosure              17
New construction             14
Pre-foreclosure / auction     8
Auction                       6
 / auction                    4
Name: status, dtype: int64
Пропусков в "status" при значениях площади в "beds": 2928 

SingleFamilyResidence    71
Single Family             2
Name: propertyType, dtype: int64
Пропусков в "propertyType" при значениях площади в "beds": 2954 

-- sqft       27
1,076 sqft     1
1,400 sqft     1
1,752 sqft     1
1,600 sqft     1
1 sqft         1
1,844 sqft     1
1,652 sqft     1
Name: sqft, dtype: int64
Пропусков в "sqft" при значениях площади в "beds": 2993 

Значения "beds" по количеству для одновременных пропусков в "status" и "propertyType"
3             202
4             132
-- sqft        95
2              91
5,000 sqft     91
             ... 
5,510 sqft      1
3.8 acres       1
7,104 sqft      1
9,443 sqft    

In [612]:
mask = data_ext['beds'].isna()
print(data[data_ext]['sqft'].value_counts(), '\n')
print(data[data_ext]['status'].value_counts(), '\n')
print(data[data_ext]['propertyType'].value_counts())

0                 10895
800 sqft            243
750 sqft            206
700 sqft            194
1,200               141
                  ...  
3,286 sqft            1
7,203                 1
5,728,968 sqft        1
16,425 sqft           1
13,870 sqft           1
Name: sqft, Length: 8984, dtype: int64 

Active                        58970
for sale                      29293
foreclosure                     703
Pending                         560
New                             454
                              ...  
Pending With Contingencies        1
Contract                          1
Pending, Continue to Show         1
Pending   Continue To Show        1
Pending Ab                        1
Name: status, Length: 62, dtype: int64 

lot/land                                                       18136
Land                                                           10744
condo                                                           6873
Traditional                                        

In [619]:
mask = (data_ext['propertyType']=='lot/land')
print(data_ext[mask]['beds'].value_counts(), '\n')

mask = (data_ext['propertyType']=='Land')
print(data_ext[mask]['beds'].value_counts(), '\n')

mask = (data_ext['propertyType']=='condo')
print(data_ext[mask]['beds'].value_counts(), '\n')

3 Beds     1027
2 Beds      660
4 Beds      437
5 Beds      126
6 Beds       72
64 Beds      44
7 Beds       18
8 Beds       14
9 Beds        7
10 Beds       3
13 Beds       2
15 Beds       1
51 Beds       1
12 Beds       1
14 Beds       1
11 Beds       1
34 Beds       1
Name: beds, dtype: int64 

0.0                     139
0                        36
3                         2
Bath                      2
4                         2
8                         1
2.0                       1
7                         1
5.0                       1
3.0                       1
1.0                       1
2                         1
1                         1
# Bedrooms 1st Floor      1
Name: beds, dtype: int64 

2 Beds     12541
3 Beds      5325
4 Beds       957
5 Beds       192
6 Beds        52
7 Beds        17
8 Beds         6
12 Beds        1
11 Beds        1
32 Beds        1
9 Beds         1
13 Beds        1
Name: beds, dtype: int64 



#### Рассмотрим признак stories

In [567]:
print('Максимальные по количеству значения встречающиехся в "stories"')
print(data_ext['stories'].value_counts()[:20], end='\n'*2)

print('Уникальных значений: {}'.format(data_ext['stories'].nunique()), end='\n'*2)

display('Уникальные значения:', data_ext['stories'].unique())

Максимальные по количеству значения встречающиехся в "stories"
1.0        67454
2.0        55283
1          23086
2          18146
3.0        11275
0.0         7241
One         5758
0           4273
3           4228
9.0         2918
2.00        2777
Two         2495
4.0         2137
2 Story     1970
1 Story     1253
1.00        1202
4            892
3+           868
6.0          718
1.50         650
Name: stories, dtype: int64

Уникальных значений: 348



'Уникальные значения:'

array([nan, '2.0', '1.0', '3.0', 'One', '2', 'Multi/Split', '4.0', '0.0',
       '0', 'One Level', '1', '9.0', '3', '1 Level, Site Built',
       'One Story', '3.00', '1.00', '14.0', 'Two', '3+', '1 Story', '5.0',
       '2 Story', 'Ranch/1 Story', 'Condominium', 'Stories/Levels', '7.0',
       '2 Level, Site Built', '2 Level', '15', '3 Level, Site Built', '4',
       '22.0', '2.00', '6.0', '1.0000', 'Lot', '3 Story', 'Three Or More',
       '1.5', '1 Level', 'Two Story or More', 'Site Built, Tri-Level',
       '54.0', '23', 'Farm House', '8.0', '16.0', '1.50', '18', '9', '21',
       '8', '12.0', 'Split Level w/ Sub', '11.0', '18.0', '1.5 Stories',
       '7', '11', 'Townhouse', '12', '21.0', '16', '1.5 Story/Basement',
       '28.0', 'Traditional', '2.5 Story', '17', '2.0000', '63.0',
       'Acreage', 'Ground Level, One', '6', 'Split Foyer', '2 Stories',
       '27.0', '19.0', '2.50', '1.30', '2 Story/Basement', 'Split Level',
       '1.5 Story', '1.5 Level', '2 Or More Stories',
  

В записях есть дополнительная информация о типе - проверим есть ли записи  о типе для этих данных по объекту (возможно запись о типе есть в признаке stories, но отсутсвует в propertyType)

In [568]:
mask_notna = data_ext['stories'].notna()
mask = data_ext[mask_notna]['stories'].apply(lambda x: True if ',' in x else False)
data_ext[mask_notna & mask]['propertyType']

133                NaN
501                NaN
585       Contemporary
610                NaN
942                NaN
              ...     
373943             NaN
374026             NaN
374716             NaN
375604             NaN
376711             NaN
Name: propertyType, Length: 631, dtype: object

In [569]:
# Выведем кол-во объектов по статусу для объектов с указание stories = 0 или 0.0
data_ext[(data_ext['stories']=='0') | (data_ext['stories']=='0.0')]['status'].value_counts()

For sale                     7264
New construction              399
Pre-foreclosure                67
Pre-foreclosure / auction      62
Foreclosure                    48
 / auction                     47
Auction                        29
Foreclosed                     22
Pending                         5
Coming soon: Dec 4.             3
Coming soon: Dec 20.            2
Coming soon: Dec 7.             2
Coming soon: Dec 6.             2
Coming soon: Dec 16.            1
Coming soon: Nov 27.            1
Coming soon: Dec 15.            1
Coming soon: Dec 14.            1
Contingent                      1
Coming soon: Nov 28.            1
Coming soon: Dec 5.             1
Coming soon: Dec 12.            1
Coming soon: Dec 9.             1
Active                          1
Coming soon: Dec 23.            1
Coming soon: Dec 13.            1
Name: status, dtype: int64

> Наблюдаются различные вариант форм записи.

> Есть записи с перечислением дополнительных данных (указание через ',' или '/'). Дополнительно имеются значения которые предназначены для признака propertyType

> Стоит отмеить значения этажей равное 0 или 0.0. Это возможно имеется ввиду объекты которые еще не построены.

В признаке "propertyType" есть записи с указание этажности здания, проверим имеются ли пропуски в "stories" для таких записей.

In [583]:
mask_isna = data_ext['stories'].isna()
mask_notna = data_ext['propertyType'].notna()
mask = data_ext[mask_notna]['propertyType'].apply(lambda x: True if ('Stories' in x) or ('Story' in x) else False)
data_ext[mask_isna & mask]['propertyType'].value_counts()

Detached, One Story                                          1613
1 Story                                                      1232
2 Stories                                                     911
Detached, Two Story                                           637
One Story                                                     592
                                                             ... 
1 Story, Spanish, Ranch, Traditional                            1
2 Stories, Traditional, Mediterranean, Texas Hill Country       1
Contemporary/Modern, Ranch, One Story                           1
1 Story, Traditional, Craftsman                                 1
3+ Stories, Contemporary, Traditional                           1
Name: propertyType, Length: 202, dtype: int64

#### Рассмотрим признак sqft

In [570]:
print('Максимальные по количеству значения встречающиехся в "sqft"')
print(data_ext['sqft'].value_counts(), end='\n'*2)

print('Уникальных значений: {}'.format(data_ext['sqft'].nunique()), end='\n'*2)

display('Уникальные значения:', data_ext['sqft'].unique())

Максимальные по количеству значения встречающиехся в "sqft"
0                                          11854
1,200 sqft                                   839
1,000 sqft                                   654
1,100 sqft                                   573
1,800 sqft                                   563
                                           ...  
9,914                                          1
Total interior livable area: 3,055 sqft        1
5,177                                          1
11620                                          1
Total interior livable area: 4,615 sqft        1
Name: sqft, Length: 25405, dtype: int64

Уникальных значений: 25405



'Уникальные значения:'

array(['2900', '1,947 sqft', '3,000 sqft', ..., '4371', '13,870 sqft',
       'Total interior livable area: 4,615 sqft'], dtype=object)

> Множество различающихся значений. Есть значения в другой размерности, но без указания единиц измерения.

> Отдельно стоит отметить значение площади 0, можно предположить, что таким образом указано отсутсвие данных о площади объекта.

In [586]:
mask = data_ext['sqft'] == '0'
data_ext[mask]['propertyType'].value_counts()

Land                             10906
Single Family Home                 489
Farms/Ranches                      178
Condo/Townhome/Row Home/Co-Op      135
Multi-Family Home                  118
Mfd/Mobile Home                     18
Other                               10
Name: propertyType, dtype: int64

In [585]:
mask = data_ext['sqft'].isna()
data_ext[mask]['propertyType'].value_counts()

lot/land                              16455
Single Family                          1411
single-family home                      839
condo                                   761
multi-family                            444
                                      ...  
Single Detached, Traditional              1
Ranch, 1 Story                            1
Bungalow, Bi-Level                        1
High Rise, Multiplex                      1
Garden Home, Spanish/Mediterranean        1
Name: propertyType, Length: 143, dtype: int64

> Для объектов с указанием площади 0 и с пропусками характерный тип land. По этому пропуски можно будет заполнить этим значением.<br/><br/>
> Стоит отметить, что такое значение площади в 0 для типа объекта land представляется очень странным, возможно, имеется ввиду что жилая площадь 0 

In [594]:
# маска тип 'lot/land'
mask = (data_ext['propertyType'] == 'lot/land')
# Выоводим площади по кличеству с фильтром маски
display(data_ext[mask]['sqft'].value_counts())

# маска тип 'Land'
mask = (data_ext['propertyType'] == 'Land')
# Выоводим площади по кличеству с фильтром маски
display(data_ext[mask]['sqft'].value_counts())

960 sqft      28
1 sqft        26
1,200 sqft    25
840 sqft      25
720 sqft      24
              ..
1,784 sqft     1
483 sqft       1
4,320 sqft     1
666 sqft       1
3,488 sqft     1
Name: sqft, Length: 2293, dtype: int64

0       10906
2665        1
948         1
1153        1
1698        1
2556        1
1470        1
3013        1
1428        1
1152        1
960         1
1596        1
Name: sqft, dtype: int64

#### Рассмотрим признак fireplace

In [571]:
print('Максимальные по количеству значения встречающиехся в "fireplace"')
print(data_ext['fireplace'].value_counts()[:20], end='\n'*2)

print('Уникальных значений: {}'.format(data_ext['fireplace'].nunique()), end='\n'*2)

display('Уникальные значения:', data_ext['fireplace'].unique())

Максимальные по количеству значения встречающиехся в "fireplace"
yes                   50356
Yes                   20856
1                     14544
2                      2432
Not Applicable         1993
Fireplace               847
3                       564
Living Room             433
LOCATION                399
Wood Burning            311
Gas/Gas Logs            300
No                      289
Fireplace YN            287
Special Features        279
1 Fireplace             274
0                       271
FAMILYRM                246
Fireplace Features      239
Great Room              207
Wood                    206
Name: fireplace, dtype: int64

Уникальных значений: 1653



'Уникальные значения:'

array(['Gas Logs', nan, 'yes', ...,
       'Ceiling Fan, Fire Sprinkler System, Storage, Utility Connection',
       'Fireplace-Gas, Fireplace-Wood, Patio, Walk-in Closet',
       'Ceiling Fan, SMAPL, Utility Connection, Walk-In Closets'],
      dtype=object)

#### Рассмотрим признак city

In [572]:
print('Максимальные по количеству значения встречающиехся в "city"')
print(data_ext['city'].value_counts()[:20], end='\n'*2)

print('Уникальных значений: {}'.format(data_ext['city'].nunique()), end='\n'*2)

display('Уникальные значения:', data_ext['city'].unique())

Максимальные по количеству значения встречающиехся в "city"
Houston            24442
San Antonio        15592
Miami              15524
Jacksonville       10015
Dallas              8856
Brooklyn            7291
Chicago             6968
New York            6919
Charlotte           6543
Las Vegas           6006
Fort Worth          5518
Orlando             5230
Austin              5155
Fort Lauderdale     5103
Cleveland           5029
Nashville           5011
Tampa               4833
Philadelphia        4797
Washington          4674
Los Angeles         4174
Name: city, dtype: int64

Уникальных значений: 2026



'Уникальные значения:'

array(['Southern Pines', 'Spokane Valley', 'Los Angeles', ...,
       'Bedford Hts', 'HAINES CITY', 'Blue Springs'], dtype=object)

#### Рассмотрим признак state

In [573]:
print('Максимальные по количеству значения встречающиехся в "state"')
print(data_ext['state'].value_counts()[:20], end='\n'*2)

print('Уникальных значений: {}'.format(data_ext['state'].nunique()), end='\n'*2)

display('Уникальные значения:', data_ext['state'].unique())

Максимальные по количеству значения встречающиехся в "state"
FL    115449
TX     83786
NY     24479
CA     23386
NC     21862
TN     18340
WA     13826
OH     12588
IL      8939
NV      8482
GA      6705
CO      6404
PA      5561
MI      5161
DC      4674
AZ      3347
IN      3328
OR      2789
MA      1516
UT      1325
Name: state, dtype: int64

Уникальных значений: 39



'Уникальные значения:'

array(['NC', 'WA', 'CA', 'TX', 'FL', 'PA', 'TN', 'IA', 'NY', 'OR', 'DC',
       'NV', 'AZ', 'GA', 'IL', 'NJ', 'MA', 'OH', 'IN', 'UT', 'MI', 'VT',
       'MD', 'CO', 'VA', 'KY', 'MO', 'WI', 'ME', 'MS', 'OK', 'SC', 'MT',
       'DE', 'Fl', 'BA', 'AL', 'OT', 'OS'], dtype=object)

#### Рассмотрим признак zipcode

In [574]:
print('Максимальные по количеству значения встречающиехся в "zipcode"')
print(data_ext['zipcode'].value_counts(), end='\n'*2)

print('Уникальных значений: {}'.format(data_ext['zipcode'].nunique()), end='\n'*2)

display('Уникальные значения:', data_ext['zipcode'].unique())

Максимальные по количеству значения встречающиехся в "zipcode"
32137         2141
33131         1563
34747         1488
78245         1390
34759         1333
              ... 
27613-4237       1
43054            1
41630            1
27615-3709       1
64015            1
Name: zipcode, Length: 4549, dtype: int64

Уникальных значений: 4549



'Уникальные значения:'

array(['28387', '99216', '90049', ..., '75792', '20191', '64015'],
      dtype=object)

In [575]:
data_ext.groupby('zipcode')['street'].count()

zipcode
--         2
0          3
00000      1
02108     23
02109     12
        ... 
99223     93
99224    128
99336    128
99337    155
99338    113
Name: street, Length: 4549, dtype: int64

> Значение zipcode как почновый индек. Можем наблюдать, что множество значений имеются для разных почтовых индексов, то есть имеем как обхекты в общих районах по почтовому индексу так и отдельные (единичные).

In [576]:
data_ext.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [545]:
# Сделаем резервную копию
data_copy_2 = data_ext

## Обработка и очистка признаков

Скорректируем записи обозначения в 'propertyType' (жаргоныые сокращения) к единому значению

In [116]:

mask_notna = data_ext['propertyType'].notna()
data_ext['propertyType'] = data_ext['propertyType'].str.lower().str.strip()

prop_types = data_ext['propertyType']
prop_types = prop_types.str.replace(',', ', ')
prop_types = prop_types.str.replace('/', ' / ')

replace_values = {
    '1': ['1-', 'one'],
    '2': ['2-', 'two'],
    '3': ['3-', 'three'],
    '4': ['4-', 'four'],
    '5': ['5-', 'five'],
    'story': ['stories']
}
for key, values in replace_values.items():
    for v in values:
        prop_types = prop_types.str.replace(v, key)

prop_types = prop_types.apply(lambda x: np.NaN if x is np.NaN else ' '.join(x.split()))
prop_types_lists = prop_types.apply(lambda x: [] if x is np.NaN else x.split(', '))

def propertyTypesRestructure(prop_types_list):
    prop_types_list_res = []
    
    mfd = [
        'manufactured home', 'manufactured house',
        'mfd', 'mo2le', 'mobile', 'mo2 le', 'mobile house',
        'modular', 'mobile home','prefab', 'mfd / mobile home'
    ]
    sfh = [
        'single family home', 'single-family home',
        'single family', 'detached', 'tiny home',
        'single detached'
    ]
    condo = [
        'condo', 'condominium'
    ]
    coop = [
        'coop', 'co-op', 'cooperative'
    ]
    town = [
        'townhome style', 'townhouse', 'townhome'
    ]
    land = [
        'land', 'lot / land'
    ]
    mfh = [
        'multi-family home', 'multi-family', 'multi family'
    ]
    
    
    for p_type in prop_types_list:
        if (p_type in mfd) or ('manufactured' in p_type):
            prop_types_list_res.append('manufactured home (house)')
        elif (p_type in sfh) or (('single' in p_type) and ('family' in p_type)):
            prop_types_list_res.append('single family home')
        elif (p_type in condo) or (('condominium' in p_type) and ('/' not in p_type)):
            prop_types_list_res.append('condo')
        elif (p_type in coop):
            prop_types_list_res.append('co-op')
        elif p_type in town or ((('townhome' in p_type) or ('townhouse' in p_type)) and ('/' not in p_type)):
            prop_types_list_res.append('townhouse')
        elif (p_type in land):
            prop_types_list_res.append('land')
        elif p_type in mfh:
            prop_types_list_res.append('multi family home')    
        elif 'other' in p_type:
            prop_types_list_res.append('other')
        elif ('multipl' in p_type) or ('multi-level' in p_type):
            prop_types_list_res.append('multi-level')
        elif ('ranch' in p_type) and ('/' not in p_type):
            prop_types_list_res.append('ranch')
        elif ('colonial' in p_type) and ('/' not in p_type):
            prop_types_list_res.append('colonial')
        else:
            prop_types_list_res.append(p_type)
    return ', '.join(set(prop_types_list_res))

d = prop_types_lists[mask_notna].apply(propertyTypesRestructure)

unique_val = []
d.apply(lambda x: unique_val.extend(x.split(', ')))
cnt = Counter(unique_val)
sorted(cnt.items(), key=lambda x: x[1], reverse=True)

[('single family home', 192845),
 ('condo', 42868),
 ('land', 31486),
 ('townhouse', 18586),
 ('multi family home', 12218),
 ('traditional', 9678),
 ('condo / townhome / row home / co-op', 7701),
 ('1 story', 4338),
 ('co-op', 3841),
 ('manufactured home (house)', 3573),
 ('ranch', 3193),
 ('2 story', 2917),
 ('contemporary', 2329),
 ('high rise', 1980),
 ('contemporary / modern', 1941),
 ('colonial', 1399),
 ('apartment', 925),
 ('other', 879),
 ('condo / townhome', 819),
 ('transitional', 780),
 ('florida', 767),
 ('bungalow', 619),
 ('spanish / mediterranean', 517),
 ('craftsman', 504),
 ('cape cod', 380),
 ('garden home', 359),
 ('mediterranean', 358),
 ('multi-level', 277),
 ('custom', 264),
 ('split-level', 224),
 ('modern', 186),
 ('cluster home', 171),
 ('split level', 162),
 ('mid-century modern', 157),
 ('spanish', 156),
 ('tudor', 152),
 ('french', 151),
 ('victorian', 135),
 ('penthouse', 134),
 ('federal', 131),
 ('', 124),
 ('key west', 121),
 ('tri-level', 118),
 ('urban

In [114]:
d = data_ext['propertyType']
mask = d[mask_notna].apply(lambda x: True if 'unknown' in x else False)

d[mask_notna & mask].value_counts()

unknown    31
Name: propertyType, dtype: int64

In [ ]:
d = data_ext['propertyType']
mask = d[mask_notna].apply(lambda x: True if 'yes' in x else False)

idx = d[mask_notna & mask].index
data_ext = data_ext.drop(idx, axis=0)

In [ ]:
d = data_ext['propertyType']
mask = d[mask_notna].apply(lambda x: True if 'unknown' in x else False)

d[mask_notna & mask].value_counts()

In [37]:
mask_notna = data_ext['propertyType'].notna()
data_ext['propertyType'] = data_ext['propertyType'].str.lower()
d = data_ext['propertyType']
d = d.str.replace('manufactured home', 'manufactured home (house)')
d = d.str.replace('manufactured house', 'manufactured home (house)')
d = d.str.replace('mfd', 'manufactured home (house)')
d = d.str.replace('mo2le', 'mobile')
d = d.str.replace('mo2 le', 'mobile')
d = d.str.replace('mobile', 'manufactured home (house)')
d = d.str.replace('prefab', 'manufactured home (house)')
d = d.str.replace('modular', 'manufactured home (house)')
d = d.str.replace('ca2 n', 'cabin')
d = d.str.replace('ca2n', 'cabin')
d = d.str.replace('mid century', 'midcentury')
d = d.str.replace('mid-century', 'midcentury')
d = d.str.replace('single-family home', 'single family')


mask = d[mask_notna].apply(lambda x: True if '/' in x and  'manufactured' in x else False)
d[mask_notna & mask].value_counts()

manufactured home (house)/manufactured                                                                            1618
manufactured home (house) / manufactured                                                                          1066
manufactured home (house)/manufactured home (house) home                                                           792
manufactured home (house), coach/carriage, manufactured home (house)                                                 2
manufactured home (house)/manufactured home (house), manufactured home (house), manufactured home (house) home       2
manuf/manufactured home (house)                                                                                      1
manuf/manufactured home (house), ranch                                                                               1
Name: propertyType, dtype: int64

### Дубликаты

In [12]:
# Посмотрим наличие полных дубликатов
mask = data_ext.duplicated(keep=False)
dubl = data_ext[mask]
print('Число записей с дублями (включительно): {}'.format(dubl.shape[0]))
# Возьмем уникальные значения улиц из набора найденных дубликатов
dubl_streets_list =  list(data[mask]['street'].unique())


Число записей с дублями (включительно): 99


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
2861,for sale,NaN,single-family home,Lexington Plan in Creeks Crossing,3 Baths,"{'atAGlanceFacts': [{'factValue': '2017', 'fac...",NaN,Lakeland,"[{'rating': ['2/10', '4/10', '2/10'], 'data': ...","3,003 sqft",33810,4 Beds,FL,1.0,NaN,NaN,NaN,"$322,900+"
25418,for sale,NaN,single-family home,Lexington Plan in Creeks Crossing,3 Baths,"{'atAGlanceFacts': [{'factValue': '2017', 'fac...",NaN,Lakeland,"[{'rating': ['2/10', '4/10', '2/10'], 'data': ...","3,003 sqft",33810,4 Beds,FL,1.0,NaN,NaN,NaN,"$322,900+"


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
7496,for sale,NaN,apartment,269 W 87th St #B,6 Baths,"{'atAGlanceFacts': [{'factValue': '2018', 'fac...",NaN,New York,"[{'rating': ['8/10'], 'data': {'Distance': ['0...","3,882 sqft",10024,5 Beds,NY,NaN,NaN,NaN,1245762,"$9,500,000"
114960,for sale,NaN,apartment,269 W 87th St #B,6 Baths,"{'atAGlanceFacts': [{'factValue': '2018', 'fac...",NaN,New York,"[{'rating': ['8/10'], 'data': {'Distance': ['0...","3,882 sqft",10024,5 Beds,NY,NaN,NaN,NaN,1245762,"$9,500,000"


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
8874,for sale,NaN,townhouse,The Lockland 27-33 Plan in Landen Pine,3.5 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Atlanta,"[{'rating': ['6/10', '6/10', '6/10'], 'data': ...","2,806 sqft",30305,4 Beds,GA,NaN,NaN,NaN,NaN,"$839,900+"
29048,for sale,NaN,townhouse,The Lockland 27-33 Plan in Landen Pine,3.5 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Atlanta,"[{'rating': ['6/10', '6/10', '6/10'], 'data': ...","2,806 sqft",30305,4 Beds,GA,NaN,NaN,NaN,NaN,"$839,900+"
172426,for sale,NaN,townhouse,The Lockland 27-33 Plan in Landen Pine,3.5 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Atlanta,"[{'rating': ['6/10', '6/10', '6/10'], 'data': ...","2,806 sqft",30305,4 Beds,GA,NaN,NaN,NaN,NaN,"$839,900+"


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
9220,for sale,NaN,lot/land,3348400805 3348400805,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Seattle,"[{'rating': ['5/10', '1/10', '3/10'], 'data': ...",NaN,98178,NaN,WA,NaN,NaN,NaN,1540940,"$111,000"
302776,for sale,NaN,lot/land,3348400805 3348400805,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Seattle,"[{'rating': ['5/10', '1/10', '3/10'], 'data': ...",NaN,98178,NaN,WA,NaN,NaN,NaN,1540940,"$111,000"


...

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
209607,foreclosure,NaN,single-family home,68 Windom Ave #A,NaN,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Staten Island,"[{'rating': ['3/10', '4/10', '5/10'], 'data': ...",600 sqft,10305,NaN,NY,NaN,NaN,NaN,NaN,NaN
214995,foreclosure,NaN,single-family home,68 Windom Ave #A,NaN,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Staten Island,"[{'rating': ['3/10', '4/10', '5/10'], 'data': ...",600 sqft,10305,NaN,NY,NaN,NaN,NaN,NaN,NaN


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
220528,for sale,NaN,condo,184 Kent Ave #A314,NaN,"{'atAGlanceFacts': [{'factValue': '1915', 'fac...",NaN,Brooklyn,"[{'rating': ['6/10'], 'data': {'Distance': ['0...",712 sqft,11249,NaN,NY,NaN,NaN,NaN,1123517,"$975,000"
226642,for sale,NaN,condo,184 Kent Ave #A314,NaN,"{'atAGlanceFacts': [{'factValue': '1915', 'fac...",NaN,Brooklyn,"[{'rating': ['6/10'], 'data': {'Distance': ['0...",712 sqft,11249,NaN,NY,NaN,NaN,NaN,1123517,"$975,000"


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
222273,for sale,NaN,single-family home,Williamsburg Plan in Creeks Crossing,2 Baths,"{'atAGlanceFacts': [{'factValue': '2017', 'fac...",NaN,Lakeland,"[{'rating': ['2/10', '4/10', '2/10'], 'data': ...","2,033 sqft",33810,3 Beds,FL,1.0,NaN,NaN,NaN,"$250,900+"
312688,for sale,NaN,single-family home,Williamsburg Plan in Creeks Crossing,2 Baths,"{'atAGlanceFacts': [{'factValue': '2017', 'fac...",NaN,Lakeland,"[{'rating': ['2/10', '4/10', '2/10'], 'data': ...","2,033 sqft",33810,3 Beds,FL,1.0,NaN,NaN,NaN,"$250,900+"


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
267262,for sale,NaN,lot/land,404 E 16th St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Winston Salem,"[{'rating': ['2/10', '5/10', '7/10'], 'data': ...",NaN,27105,NaN,NC,NaN,NaN,NaN,682831,"$8,000"
353154,for sale,NaN,lot/land,404 E 16th St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Winston Salem,"[{'rating': ['2/10', '5/10', '7/10'], 'data': ...",NaN,27105,NaN,NC,NaN,NaN,NaN,682831,"$8,000"


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
286380,for sale,NaN,lot/land,607 Kimpton Ct,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Greensboro,"[{'rating': ['4/10', '6/10', '4/10'], 'data': ...",NaN,27455,NaN,NC,NaN,NaN,NaN,914331,"$90,000"
345644,for sale,NaN,lot/land,607 Kimpton Ct,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Greensboro,"[{'rating': ['4/10', '6/10', '4/10'], 'data': ...",NaN,27455,NaN,NC,NaN,NaN,NaN,914331,"$90,000"


In [ ]:
dublicate = data_ext.duplicated()
print('Кол-во дубликатов: {}'.format(data_ext[dublicate].shape[0]))
data_ext = data_ext.drop_duplicates(ignore_index=True)

### Признак 'status'

In [206]:
data_ext['status'] = data_ext['status'].str.lower().str.strip()
data_ext['status'] = data_ext['status'].apply(lambda x: np.NaN if x is np.NaN else ' '.join(x.split()))

In [207]:
mask_notna = data_ext['status'].notna()

In [219]:
mask_1 = data_ext[mask_notna]['status'].apply(lambda x: True if 'with offer' in x else False)
mask_2 = data_ext[mask_notna]['status'].apply(lambda x: True if 'auction' in x else False)
mask_3 = data_ext[mask_notna]['status'].apply(lambda x: True if 'pre' in x else False)
mask_4 = data_ext[mask_notna]['status'].apply(lambda x: True if 'active' in x else False)
mask_5 = data_ext[mask_notna]['status'].apply(lambda x: True if ('bckp' in x) | ('backup' in x) | ('back up' in x) else False)
mask_6 = data_ext[mask_notna]['status'].apply(lambda x: True if 'pending' in x else False)

In [222]:
mask_sold = data_ext[mask_notna]['status'].apply(lambda x: True if 'sold' in x else False)
data_ext[mask_notna & mask_sold]['status'].value_counts()

recently sold     287
sold: $285,000      1
sold: $15,000       1
Name: status, dtype: int64

In [ ]:
mask = d['status'].notna()
msk = d[mask]['status'].apply(lambda x: True if ' - ' in x else False)
d[mask & msk]['status'].unique()

In [ ]:
status_list = [
    'active', 
    'pending', 
    'pending continue show',
    'pending take backups',
    'pending backups wanted',
    'pending inspection',
    'pending with contingencies',
    
]

In [306]:
def reStr(x):
    if x is np.NaN:
        return np.NaN
    
    #x = x.split('/')
    # for
    
    x = x
    x_l = x.split()
    
    cond_back = ('bckp' in x) or ('backup' in x) or ('back up' in x)
    cond_insp = ('inspection' in x) or ('insp' in x)
    
    if ('pending' in x) or ('p' in x_l) or ('pi' in x_l): 
        if ('pending' == x) \
        or ('pending sale' in x) or ('offer pending signature' in x) \
        or ('option pending' in x) or ('pending ab' in x) \
        or ('pending offer approval' in x):
            return 'pending'
        elif ('sh' in x_l) or ('show' in x):
            return 'pending continue show'
        elif ('backup offer requested' in x) or ('bring backup' in x) \
        or ((('taking' in x) or ('take' in x)) and (cond_back)):
            return 'pending take backups'
        elif ('want' in x) and (cond_back):
            return 'pending backups wanted'
        elif (cond_insp) or ('in' in x_l) or ('pi' in x_l):
            return 'pending inspection'
        elif ('with contingencies' in x) or ('cont.' in x_l) \
        or ('cont' in x_l) or ('w/' in x):
            return 'pending with contingencies'
        else: return x
    elif cond_insp:
        return 'insp inspection contingency'
    else: return x
        
        
s = data_ext['status'].apply(reStr)

In [307]:
s[mask_notna & mask_6].value_counts()

pending                       4980
pending continue show          208
pending take backups            84
pending with contingencies      47
pending inspection              37
lease/purchase pending           3
pending fe                       3
pending backups wanted           2
Name: status, dtype: int64

<hr/>

In [201]:
def reStr(x):
    if x is np.NaN:
        return np.NaN
    
    if (('under contract' in x) or ('u' in x.split()) or ('uc' in x.split()))  and ('no show' not in x): 
        return 'under contract showing' 
    elif ('bckp' in x) or ('backup' in x) or ('back up' in x):
        if ('contract' in x) or ('active' in x):
            return 'under contract backups'
        elif 'pending' in x:
            if (('taking' in x) or ('take' in x)) and ('back' in x):
                return 'pending take backups'
            if 'want' in x:
                return 'pending backup wanted'
        else: return x
    elif 'ps' in x.split():
        return 'pending continue show'
    elif ('c' in x.split()) or ('pending' in x):
        if (('c' in x.split()) or ('continue show' in x) or ('continue to show') or ('sh' in x.split())):
            return 'pending continue show'
    elif ('pi' in x.split()):
        return 'pending inspection'
    elif ('inspection' in x) or ('insp' in x):
        if 'pending' in x:
            return 'pending inspection'
        else:
            return 'insp inspection contingency'
    elif ('contingency' in x) or ('contingencies' in x):
        if 'active' in x:
            return 'active contingency'
        elif 'contract' in x:
            return 'contingency contract'
        else: return x   
    elif 'new' in x:
        return 'new construction'
    elif 'escape' in x:
        return 'pending escape clause'
    elif ('foreclosure' in x) or ('foreclosed' in x) or ('pf' in x.split()):
        if 'pre' in x:
            return 'pre-foreclosure'
        else: return 'foreclosure'
    elif ('auction' in x) and ('foreclos' not in x):
        return 'auction'
    elif 'coming soon' in x:
        return 'coming soon'
    elif (x == 'acitve') or (('a' in x.split()) and ('active' in x)):
        return 'active'
    elif 'rent' in x:
        return 'for rent'
    elif 'sold' in x:
        return 'sold'
    else: 
        return x
    
        

s = data_ext['status'].apply(reStr)

In [40]:
dict_sim = {
    'active': ['a active', 'acitve'],
    'active with contract': ['active with contract'],
    'active with offer': ['active with offer'],
    'auction': ['auction - active', 'active - auction', 'auction'],
    'under contract showing': ['active under contract', 'under contract showing', 'under contract show', 'under contract'],
    'under contract backups': ['active backup', 'backup contract', 'under contract backups'],
    'active contingency': ['conditional', 'active with contingencies', 'active contingency', 'active/contingent', 'active contingent'],
    'pre foreclosure' : ['pre foreclosure', 'pre-foreclosure / auction', 'pre-foreclosure']
    'under contract': ['u under contract', 'under contract']
}


In [41]:
mask_notna = data_ext['status'].notna()

In [42]:
temp = data_ext[mask_notna]['status']
for key, values in dict_sim.items():
    temp = temp.str.lower().apply(lambda x: key if x in values else x)

In [43]:
temp.value_counts()

for sale                199571
active                  105650
foreclosure               6769
new construction          5475
pending                   4807
                         ...  
coming soon: oct 24.         1
coming soon: oct 21.         1
pending take backups         1
contract                     1
coming soon: dec 23.         1
Name: status, Length: 141, dtype: int64

In [35]:
data_ext[mask_notna]['status'].apply(lambda x: x if 'active' in x else None).value_counts()

active                       105207
active under contract           718
a active                        443
active with offer                49
active backup                    39
auction - active                 28
active/contingent                15
active with contingencies        13
active - auction                 10
active contingency                8
active option                     7
active option contract            7
active contingent                 6
active - contingent               2
active with contract              2
temporary active                  1
Name: status, dtype: int64

In [ ]:
data_ext = data_ext.drop('street', axis=1)

<hr/>

ДЛЯ ПРОДОЛЖЕНИЯ

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8), sharex=True)

histplot = sns.histplot(
    x = data_ext['target'],
    bins=20,
    ax = ax[0]
)
histplot.set_title('Распределение цены', size=16)
histplot.set_xlabel('Стоимость', size=12)
histplot.set_ylabel('Количество', size=12)

fig.show()


_, p = stats.normaltest(data_ext['target'])
print('p = {}'.format(p))
print('stat = {}'.format(_))
# Интерпретация 
if p > 0.05:
	print('Данные распрделенны нормально')
else:
	print('Данные не распрделенны нормально')

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(8,8))

scatterplot = sns.scatterplot(
    data=data_ext,
    x=data_ext['sqft'],
    y=data_ext['stories'],
    how=data_ext['target'],
    ax=ax[0]
)
scatterplot.set_title('Цена от этажей и площади', size=16)
scatterplot.set_xlabel('Площадь', size=12)
scatterplot.set_ylabel('Этажи', size=12)

scatterplot = sns.scatterplot(
    data=data_ext,
    x=data_ext['sqft'],
    y=data_ext['Year bild'],
    how=data_ext['target'],
    ax=ax[0]
)
scatterplot.set_title('Цена от возраста и площади', size=16)
scatterplot.set_xlabel('Площадь', size=12)
scatterplot.set_ylabel('возраста объекта', size=12)

plt.subplots_adjust(hspace=0.5)

fig.show()

In [ ]:
# постройте корреляционную матрицу и оцените данные на предмет наличия мультиколлинеарности
corr_matrix = data_ext.corr()
fig, ax = plt.subplots(1,1,figsize=(12,10))
heatmap = sns.heatmap(
    corr_matrix,
    ax = ax
)
fig.show()

In [ ]:
mask_func_corr = lambda x: ((x > 0.75) | (x < -0.75)) & (x!=1)

corr = corr_matrix.unstack().reset_index()
corr = corr.rename(columns={0:'corr'})
corr = corr[corr['corr'].apply(mask_func_corr)]
corr['level_0'], corr['level_1'] = corr[['level_0','level_1']].min(axis=1), corr[['level_0','level_1']].max(axis=1)
corr.drop_duplicates()

In [ ]:
X=None
y=None

In [2]:
SEED = np.random.seed(42)

In [ ]:
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(
    X, y, 
    test_size=0.3
)

In [ ]:
feature=None
selector = feature_selection.SelectKBest(feature_selection.f_regression, k=25)
selector.fit(X_train[feature], y_train)
best_features = selector.get_feature_names_out()

feature_a=None
selector = feature_selection.SelectKBest(feature_selection.f_classif, k=25)
selector.fit(X_train[feature_a], y_train)
best_features_a = selector.get_feature_names_out()

print('Отобранные признаки методом отбора на основе фильтров')
print(*best_features, sep='\n')
print(*best_features_a, sep='\n')

X_train = X_train[best_features]
X_valid = X_valid[best_features]

In [ ]:
# нормализуйте данные с помощью minmaxsxaler
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
columns_name = X_train.columns
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_valid)
print(np.mean(X_train[:,0]))
print(np.mean(X_test[:,0]))
X_train = pd.DataFrame(X_train, columns=columns_name)
X_test = pd.DataFrame(X_test, columns=columns_name)
display(X_train.head(3), X_test.head(3))

In [ ]:
models = dict()

lr = linear_model.LinearRegression()
lr.fit(X_train, y_train) 
y_train_pred = lr.predict(X_train)
y_valid_pred = lr.predict(X_valid)

mape_train = round(np.sqrt(metrics.mean_absolute_percentage_error(y_train, y_train_pred)),2)
mape_valid = round(np.sqrt(metrics.mean_absolute_percentage_error(y_valid, y_valid_pred)),2)
models['lr'] = [mape_train, mape_valid]

print('MAPE на тренировочных данных: {}'.format(mape_train))
print('MAPE на валидационных данных: {}'.format(mape_valid))

In [ ]:
dtr = tree.DecisionTreeRegressor()
dtr.fit(X_train, y_train) 
y_train_pred = dtr.predict(X_train)
y_valid_pred = dtr.predict(X_valid)

mape_train = round(np.sqrt(metrics.mean_absolute_percentage_error(y_train, y_train_pred)),2)
mape_valid = round(np.sqrt(metrics.mean_absolute_percentage_error(y_valid, y_valid_pred)),2)
models['dtr'] = [mape_train, mape_valid]

print('MAPE на тренировочных данных: {}'.format(mape_train))
print('MAPE на валидационных данных: {}'.format(mape_valid))

In [ ]:
result_train = []
result_valid = []

max_depths = range(7, 20)

for depth in max_depths:
    dtr = tree.DecisionTreeRegressor(max_depth=depth)
    dtr.fit(X_train, y_train) 
    y_train_pred = dtr.predict(X_train)
    y_valid_pred = dtr.predict(X_valid)

    mape_train = round(np.sqrt(metrics.mean_absolute_percentage_error(y_train, y_train_pred)),2)
    mape_valid = round(np.sqrt(metrics.mean_absolute_percentage_error(y_valid, y_valid_pred)),2)
    result_train.append(mape_train)
    result_valid.append(mape_valid)


fig , ax  = plt.subplots(1, 1, figsize=(8,6))

plot = sns.lineplot(
    x=max_depths,
    y=result_train,
    label='train',
    ax=ax
)
plot2 = sns.lineplot(
    x=max_depths,
    y=result_valid,
    label='valid',
    ax=ax
)

fig.show()

In [ ]:
rf = ensemble.RandomForestRegressor(
    n_estimators=200, 
    max_depth=12,
    criterion='squared_error',
    min_samples_split=20
)

rf.fit(X_train, y_train) 
y_train_pred = rf.predict(X_train)
y_valid_pred = rf.predict(X_valid)

mape_train = round(np.sqrt(metrics.mean_absolute_percentage_error(y_train, y_train_pred)),2)
mape_valid = round(np.sqrt(metrics.mean_absolute_percentage_error(y_valid, y_valid_pred)),2)
models['rf'] = [mape_train, mape_valid]

print('MAPE на тренировочных данных: {}'.format(mape_train))
print('MAPE на валидационных данных: {}'.format(mape_valid))

In [ ]:
gbr = ensemble.GradientBoostingRegressor(
    learning_rate=0.5,
    n_estimators=100, 
    max_depth=6,
    min_samples_split=30,
    random_state=42
)

gbr.fit(X_train, y_train) 
y_train_pred = gbr.predict(X_train)
y_valid_pred = gbr.predict(X_valid)

mape_train = round(np.sqrt(metrics.mean_absolute_percentage_error(y_train, y_train_pred)),2)
mape_valid = round(np.sqrt(metrics.mean_absolute_percentage_error(y_valid, y_valid_pred)),2)
models['gbr'] = [mape_train, mape_valid]

print('MAPE на тренировочных данных: {}'.format(mape_train))
print('MAPE на валидационных данных: {}'.format(mape_valid))

In [ ]:
# подберите оптимальные параметры с помощью gridsearch
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth': list(range(3, 50))}
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    n_jobs=-1,
    cv=5
)
grid_search.fit(X_train, y_train) 

print('accuracy на обучающей выборке: {:.4f}'.format(grid_search.score(X_train, y_train)))
print('accuracy на тестовой выборке: {:.4f}'.format(grid_search.score(X_test, y_test)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

In [ ]:
import optuna

def optuna_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
    max_depth = trial.suggest_int('max_depth', 10, 30, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

    # создаем модель
    model = ensemble.RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    # обучаем модель
    model.fit(X_train, y_train)
    score = metrics.f1_score(y_train, model.predict(X_train))
    
    return score

study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)